In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from helpers.utils import creat_datasets, creating_val_data, reset_graph, grid_serach, read_pickle_file, write_pickle_file
from helpers.utils import creating_train_val_test_datasets, tensorboard_callback,  save_best_model_callback
from helpers.bln_layer import  bln_layer
from helpers.dense_layer import  dense_layer
from helpers.bln_callback import bln_callback 




from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [2]:
tf.config.experimental_run_functions_eagerly(True)

In [3]:
tf.__version__

'2.1.0'

### Loading data

In [4]:
random_seed = 100
minibatch = 25
buffersize = 60000
number_valid_sampels = 5000 # number of validation data
epochs = 5
learning_rate = 0.005

#((25000  - 5000 )/ 25 ) * 1
number_batches_train = 800 # number of batches to train, each batch of size minibatch parameter
number_batches_valid = 200 # number of batches to validate, each batch of size minibatch parameter (( 5000 )/ 25 ) * 1
num_classes = 1

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)


In [5]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...


C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [6]:
x_train, y_train, x_valid, y_valid = creating_val_data(x_train, y_train,
                                                       number_valid_sampels = number_valid_sampels,
                                                       random_seed=random_seed)

train_dataset, valid_dataset, test_dataset = creating_train_val_test_datasets(x_train, y_train,
                                                                              x_test, y_test,
                                                                              x_valid, y_valid, 
                                                                              minibatch = minibatch,
                                                                              buffersize= buffersize,
                                                                              random_seed=random_seed)

In [7]:
train_dataset

<PrefetchDataset shapes: ((25, 80), (25,)), types: (tf.int32, tf.int64)>

# 1. Using Batch Layer Normalization Layer

In [8]:
def BLNLayer_model(inputshape= (80), max_features = 20000, embed_size=256, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1,
                          batch_size = 10,
                          b_mm = True, b_mv = True,
                          f_mm = False, f_mv = False):
    
   
    # building the model
  
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn1', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn2', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn3', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x)
           
    
    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])

In [9]:
model_bln_layer = BLNLayer_model(inputshape= (80), max_features = max_features, 
                                        embed_size= 256, random_seed = random_seed,
                                        lstm_unit = 128 , dense_units= num_classes,
                                        batch_size = minibatch,
                                        b_mm = False, b_mv = False,
                                        f_mm = False, f_mv = False
                                        )



In [10]:
model_bln_layer.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(25, 80)]                0         
_________________________________________________________________
embedding (Embedding)        (25, 80, 256)             5120000   
_________________________________________________________________
lstm (LSTM)                  (25, 80, 128)             197120    
_________________________________________________________________
bn1 (bln_layer)              (25, 80, 128)             24738     
_________________________________________________________________
lstm_1 (LSTM)                (25, 64)                  49408     
_________________________________________________________________
bn2 (bln_layer)              (25, 64)                  308       
_________________________________________________________________
dense1 (dense_layer)         (25, 32)                  2080  

# callbacks

In [11]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/bln_layer_imdb_wohletraining_batchSize_' + str(minibatch) 
save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

# Callback for resetting moving mean and variances at the end of each epoch
bln_layer_cb = bln_callback()

bln_layer_cb_list = [save_bm_cb, tb_cb, bln_layer_cb]

In [12]:
model_bln_layer.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                               loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                               metrics = [tf.keras.metrics.BinaryAccuracy()])

In [13]:
model_bln_layer_history =  model_bln_layer.fit(train_dataset.take(number_batches_train), epochs=epochs,
                                                verbose=1, callbacks=bln_layer_cb_list,
                                                validation_data=valid_dataset.take(number_batches_valid),
                                                shuffle=True)

Train for 800 steps, validate for 200 steps
Epoch 1/5
800/800 [==============================] - 678s 848ms/step - loss: 0.4811 - binary_accuracy: 0.7471 - val_loss: 0.3794 - val_binary_accuracy: 0.8182
Epoch 2/5
800/800 [==============================] - 684s 855ms/step - loss: 0.2622 - binary_accuracy: 0.8903 - val_loss: 0.4469 - val_binary_accuracy: 0.8134
Epoch 3/5
800/800 [==============================] - 690s 863ms/step - loss: 0.1549 - binary_accuracy: 0.9394 - val_loss: 0.5332 - val_binary_accuracy: 0.8132
Epoch 4/5
800/800 [==============================] - 690s 862ms/step - loss: 0.1011 - binary_accuracy: 0.9600 - val_loss: 0.7195 - val_binary_accuracy: 0.8110
Epoch 5/5
800/800 [==============================] - -85723s -107153598us/step - loss: 0.0766 - binary_accuracy: 0.9712 - val_loss: 0.7969 - val_binary_accuracy: 0.8072


In [14]:
del model_bln_layer, save_bm_cb, tb_cb, bln_layer_cb, bln_layer_cb_list
reset_graph()

session is clear


In [15]:
# Select the model with the lowest validation loss
weights_path = './models/rnn/bln_layer_imdb_wohletraining_batchSize_25/best_model.h5'

In [16]:
save_eval_path = "./logs/" + folder_name + '/'+ str(number_batches_train) +"_sorted_evaluation.pkl"
evaluation = grid_serach(BLNLayer_model, test_dataset,
                         batch_size = minibatch, sort=True,
                         save_eval_path = save_eval_path,
                         weights_path = weights_path,
                         loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                         metrics = tf.keras.metrics.BinaryAccuracy())



1000/1000 [==============================] - 306s 306ms/step - loss: 0.3763 - binary_accuracy: 0.8426
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.37629663967341187, 0.8426]}
session is clear
1000/1000 [==============================] - 306s 306ms/step - loss: 0.3756 - binary_accuracy: 0.8426
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.37629663967341187, 0.8426], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.37557747545838355, 0.84256]}
session is clear
1000/1000 [==============================] - 304s 304ms/step - loss: 0.3762 - binary_accuracy: 0.8426
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.37629663967341187, 0.8426], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.37557747545838355, 0.84256], 'Bmm_True Bmv_True Fmm_False Fmv_True': [0.3762035561949015, 0.84256]}
session is clear
1000/1000 [==============================] - 303s 303ms/step - loss: 0.3755 - binary_accuracy: 0.8425
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.37629663967341187, 0.8426], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0

1000/1000 [==============================] - 318s 318ms/step - loss: 0.3717 - binary_accuracy: 0.8262
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.37629663967341187, 0.8426], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.37557747545838355, 0.84256], 'Bmm_True Bmv_True Fmm_False Fmv_True': [0.3762035561949015, 0.84256], 'Bmm_True Bmv_True Fmm_False Fmv_False': [0.3755008724406362, 0.84248], 'Bmm_True Bmv_False Fmm_True Fmv_True': [0.4407207244038582, 0.8286], 'Bmm_True Bmv_False Fmm_True Fmv_False': [0.4403442090898752, 0.82864], 'Bmm_True Bmv_False Fmm_False Fmv_True': [0.4404822921603918, 0.82868], 'Bmm_True Bmv_False Fmm_False Fmv_False': [0.44014124329388143, 0.82872], 'Bmm_False Bmv_True Fmm_True Fmv_True': [0.38330538057535884, 0.82912], 'Bmm_False Bmv_True Fmm_True Fmv_False': [0.3829549258388579, 0.8292], 'Bmm_False Bmv_True Fmm_False Fmv_True': [0.38327607592567803, 0.82908], 'Bmm_False Bmv_True Fmm_False Fmv_False': [0.3829385451301932, 0.8292], 'Bmm_False Bmv_False Fmm_True Fmv_Tr

In [18]:
evaluation 

[('Bmm_False Bmv_False Fmm_True Fmv_True', [0.37166605739295483, 0.82616]),
 ('Bmm_False Bmv_False Fmm_False Fmv_True', [0.37168192461133004, 0.82616]),
 ('Bmm_False Bmv_False Fmm_True Fmv_False', [0.3717609993070364, 0.82636]),
 ('Bmm_False Bmv_False Fmm_False Fmv_False', [0.37177927139401434, 0.8262]),
 ('Bmm_True Bmv_True Fmm_False Fmv_False', [0.3755008724406362, 0.84248]),
 ('Bmm_True Bmv_True Fmm_True Fmv_False', [0.37557747545838355, 0.84256]),
 ('Bmm_True Bmv_True Fmm_False Fmv_True', [0.3762035561949015, 0.84256]),
 ('Bmm_True Bmv_True Fmm_True Fmv_True', [0.37629663967341187, 0.8426]),
 ('Bmm_False Bmv_True Fmm_False Fmv_False', [0.3829385451301932, 0.8292]),
 ('Bmm_False Bmv_True Fmm_True Fmv_False', [0.3829549258388579, 0.8292]),
 ('Bmm_False Bmv_True Fmm_False Fmv_True', [0.38327607592567803, 0.82908]),
 ('Bmm_False Bmv_True Fmm_True Fmv_True', [0.38330538057535884, 0.82912]),
 ('Bmm_True Bmv_False Fmm_False Fmv_False', [0.44014124329388143, 0.82872]),
 ('Bmm_True Bmv_Fals